# What is blockedonweibo?

This python package allows you to automate tests to check if a keyword is censored or not on Sina Weibo, a Chinese social media site. It is an updated version of the script which was used to detect keywords on the site, http://blockedonweibo.com. It handles interrupted tests, multiple tests per day, storing results to a database, and a number of other features which simplify testing Weibo censorship at scale. The researcher merely has to feed the script a list of words for one-off tests. For recurring tests, simply wrap the script with a scheduler.

<img src="../screenshot.png" alt="screenshot" style="width: 700px;"/>

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#What-is-blockedonweibo?" data-toc-modified-id="What-is-blockedonweibo?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is blockedonweibo?</a></div><div class="lev1 toc-item"><a href="#Install-the-blockedonweibo-package" data-toc-modified-id="Install-the-blockedonweibo-package-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Install the blockedonweibo package</a></div><div class="lev1 toc-item"><a href="#Adjust-your-settings" data-toc-modified-id="Adjust-your-settings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Adjust your settings</a></div><div class="lev1 toc-item"><a href="#Let's-start-testing!" data-toc-modified-id="Let's-start-testing!-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Let's start testing!</a></div><div class="lev2 toc-item"><a href="#Pass-a-dictionary-of-keywords-to-start-testing" data-toc-modified-id="Pass-a-dictionary-of-keywords-to-start-testing-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Pass a dictionary of keywords to start testing</a></div><div class="lev2 toc-item"><a href="#Pass-in-cookies-so-you-can-also-get-the-number-of-results.-Pass-in-sqlite_file-to-save-the-results-to-disk-so-you-can-load-it-later" data-toc-modified-id="Pass-in-cookies-so-you-can-also-get-the-number-of-results.-Pass-in-sqlite_file-to-save-the-results-to-disk-so-you-can-load-it-later-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Pass in cookies so you can also get the number of results. Pass in sqlite_file to save the results to disk so you can load it later</a></div><div class="lev2 toc-item"><a href="#If-your-test-gets-interrupted-or-you-add-more-keywords,-you-can-pick-up-where-you-left-off" data-toc-modified-id="If-your-test-gets-interrupted-or-you-add-more-keywords,-you-can-pick-up-where-you-left-off-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>If your test gets interrupted or you add more keywords, you can pick up where you left off</a></div><div class="lev2 toc-item"><a href="#You-can-attach-notes-or-categorizations-to-your-keywords-for-easy-querying-and-analysis-later" data-toc-modified-id="You-can-attach-notes-or-categorizations-to-your-keywords-for-easy-querying-and-analysis-later-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>You can attach notes or categorizations to your keywords for easy querying and analysis later</a></div><div class="lev2 toc-item"><a href="#If-you-want-to-test-multiple-times-a-day,-just-pass-in-the-test_number-param" data-toc-modified-id="If-you-want-to-test-multiple-times-a-day,-just-pass-in-the-test_number-param-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>If you want to test multiple times a day, just pass in the <code>test_number</code> param</a></div><div class="lev2 toc-item"><a href="#It-can-skip-redundant-keywords" data-toc-modified-id="It-can-skip-redundant-keywords-46"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>It can skip redundant keywords</a></div><div class="lev2 toc-item"><a href="#You-can-also-pass-in-lists-if-you-prefer-(though-you-can't-include-the-source-or-notes)" data-toc-modified-id="You-can-also-pass-in-lists-if-you-prefer-(though-you-can't-include-the-source-or-notes)-47"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>You can also pass in lists if you prefer (though you can't include the source or notes)</a></div>

# Install the blockedonweibo package

The github repo for this Weibo keyword testing script is located at https://github.com/jasonqng/blocked-on-weibo.

To begin using this python package, inside your terminal, do a
```
git clone https://github.com/jasonqng/blocked-on-weibo.git
```
or 

```
git clone git@github.com:jasonqng/blocked-on-weibo.git
``` 
(if you prefer ssh).

Then `cd` into the downloaded directory and install the requirements then the package:

```
pip install -r requirements.txt
python setup.py install
```

To confirm the installation works, in a python shell (you can start by running `python` from terminal), try importing the package:

```
import blockedonweibo
```

If you don't get any errors, things have installed successfully. If not, you may need to fiddle with your python paths and settings to ensure it's being installed to the correct location.

# Adjust your settings

Your python script only requires the following. All other imports are handled by the package.

In [1]:
from blockedonweibo import weibo
import pandas as pd

You have the option of saving your test results to a file. You'll need to pass a path to to a file which will store the results in sqlite format. It can be helpful to set this at the top of your script and pass the variable each time you run the test.

In [2]:
sqlite_file = 'results.sqlite' # name of sqlite file to read from/write to

If you want to erase any existing data you have in the sqlite file defined above, just pass overwrite=True to the `create_database` function. Otherwise any new results will be appended to the end of the database.

In [3]:
weibo.create_database(sqlite_file, overwrite=True)

This testing script is enhanced if you allow it to log into Weibo, which increases your rate limit threshold as well as returns the number of results a search says it has. This script will work without your supplying credentials, but it is highly recommended. To do so, edit the `weibo_credentials.py` with your email address and password. The file is ignored and will not be uploaded by default when you push commits to github. You can inspect the code to verify that the credentials don't go anywhere except to weibo.

Using those credentials, the script logs you in and fetches a cookie for the user session you create. This cookie can be saved to a file by passing the `write_cookie` parameter in the `user_login` function.

In [4]:
session = weibo.user_login(write_cookie=True)

There is a helper function to verify that the cookie actually works

In [5]:
cookie = session.cookies.get_dict()
print(weibo.verify_cookies_work(cookie))

True


If you have the cookie already written to disk, you don't need to perform another `user_login` and instead, you can just use the `load_cookies` function to fetch the cookie from the file. Again, you can verify that it works. Just store the cookie's contents (a dictionary) to a variable and pass that to the `run` function below if you want to test as if you were logged in. Otherwise, it will emulate a search by a logged out user.

In [6]:
cookie = weibo.load_cookies()
print(weibo.verify_cookies_work(cookie))

True


# Let's start testing!

## Pass a dictionary of keywords to start testing

In [7]:
sample_keywords_df = pd.DataFrame(
    [{'keyword':'hello','source':'my dataframe'},
     {'keyword':'lxb','source':'my dataframe'},
     {'keyword':u'习胞子','source':'my dataframe'}
    ])

In [8]:
sample_keywords_df

,keyword,source
0,hello,my dataframe
1,lxb,my dataframe
2,习胞子,my dataframe


In [9]:
weibo.run(sample_keywords_df,insert=False,return_df=True)

(0, u'hello', 'has_results')
(1, u'lxb', 'censored')
(2, u'\u4e60\u80de\u5b50', 'no_results')


,date,datetime,keyword,num_results,result,source,test_number
0,2017-09-25,2017-09-25 10:12:45.280812,hello,[],has_results,my dataframe,1
0,2017-09-25,2017-09-25 10:13:00.191900,lxb,None,censored,my dataframe,1
0,2017-09-25,2017-09-25 10:13:16.356805,习胞子,None,no_results,my dataframe,1


## Pass in cookies so you can also get the number of results. Pass in sqlite_file to save the results to disk so you can load it later

In [10]:
weibo.run(sample_keywords_df,sqlite_file=sqlite_file,cookies=cookie)

(0, u'hello', 'has_results')
(1, u'lxb', 'censored')
(2, u'\u4e60\u80de\u5b50', 'no_results')


In [11]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None


## If your test gets interrupted or you add more keywords, you can pick up where you left off

Let's pretend I wanted to test four total keywords, but I was only able to complete the first three above. I'll go ahead and add one more keyword to the test list to replicate an unfinished keyword.

In [12]:
sample_keywords_df.loc[len(sample_keywords_df.index)] = ['刘晓波','my dataframe']

In [13]:
sample_keywords_df

,keyword,source
0,hello,my dataframe
1,lxb,my dataframe
2,习胞子,my dataframe
3,刘晓波,my dataframe


In [14]:
weibo.run(sample_keywords_df,sqlite_file=sqlite_file,cookies=cookie)

(3, u'\u5218\u6653\u6ce2', 'censored')


Neat-o, it was smart enough to start right at that new keyword and not start all over again!

In [15]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None
3,3,2017-09-25,2017-09-25 10:14:29.667395,1,刘晓波,0,0,0,censored,my dataframe,NaN,None


## You can attach notes or categorizations to your keywords for easy querying and analysis later

In [16]:
new_keywords_df = pd.DataFrame(
    [{'keyword':'pokemon','source':'my dataframe',"notes":"pop culture"},
     {'keyword':'jay chou','source':'my dataframe',"notes":"pop culture"},
     {'keyword':u'weibo','source':'my dataframe',"notes":"social media"}
    ])
merged_keywords_df = pd.concat([sample_keywords_df,new_keywords_df]).reset_index(drop=True)
merged_keywords_df

,keyword,notes,source
0,hello,NaN,my dataframe
1,lxb,NaN,my dataframe
2,习胞子,NaN,my dataframe
3,刘晓波,NaN,my dataframe
4,pokemon,pop culture,my dataframe
5,jay chou,pop culture,my dataframe
6,weibo,social media,my dataframe


In [17]:
weibo.run(merged_keywords_df,sqlite_file=sqlite_file,cookies=cookie)

(4, u'pokemon', 'has_results')
(5, u'jay chou', 'has_results')
(6, u'weibo', 'has_results')


In [18]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None
3,3,2017-09-25,2017-09-25 10:14:29.667395,1,刘晓波,0,0,0,censored,my dataframe,NaN,None
4,4,2017-09-25,2017-09-25 10:14:49.107078,1,pokemon,0,0,0,has_results,my dataframe,5705260.0,pop culture
5,5,2017-09-25,2017-09-25 10:15:09.762484,1,jay chou,0,0,0,has_results,my dataframe,881.0,pop culture
6,6,2017-09-25,2017-09-25 10:15:28.100418,1,weibo,0,0,0,has_results,my dataframe,63401495.0,social media


In [19]:
results = weibo.sqlite_to_df(sqlite_file)
results.query("notes=='pop culture'")

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
4,4,2017-09-25,2017-09-25 10:14:49.107078,1,pokemon,0,0,0,has_results,my dataframe,5705260.0,pop culture
5,5,2017-09-25,2017-09-25 10:15:09.762484,1,jay chou,0,0,0,has_results,my dataframe,881.0,pop culture


In [20]:
results.query("notes=='pop culture'").num_results.mean()

2853070.5

## If you want to test multiple times a day, just pass in the `test_number` param

You can off `verbose` output in case you don't need to troubleshoot anything...

In [21]:
weibo.run(sample_keywords_df,sqlite_file=sqlite_file,cookies=cookie,verbose='none',test_number=2)

In [22]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None
3,3,2017-09-25,2017-09-25 10:14:29.667395,1,刘晓波,0,0,0,censored,my dataframe,NaN,None
4,4,2017-09-25,2017-09-25 10:14:49.107078,1,pokemon,0,0,0,has_results,my dataframe,5705260.0,pop culture
5,5,2017-09-25,2017-09-25 10:15:09.762484,1,jay chou,0,0,0,has_results,my dataframe,881.0,pop culture
6,6,2017-09-25,2017-09-25 10:15:28.100418,1,weibo,0,0,0,has_results,my dataframe,63401495.0,social media
7,7,2017-09-25,2017-09-25 10:15:46.214464,2,hello,0,0,0,has_results,my dataframe,80454634.0,None
8,8,2017-09-25,2017-09-25 10:16:03.274804,2,lxb,0,0,0,censored,my dataframe,NaN,None
9,9,2017-09-25,2017-09-25 10:16:19.035805,2,习胞子,0,0,0,no_results,my dataframe,NaN,None


## It can skip redundant keywords

In [23]:
more_keywords_df = pd.DataFrame(
    [{'keyword':'zhongnanhai','source':'my dataframe2',"notes":"location"},
     {'keyword':'cats','source':'my dataframe2',"notes":"pop culture"},
     {'keyword':'zhongnanhai','source':'my dataframe2',"notes":"location"}
    ])

In [24]:
more_keywords_df

,keyword,notes,source
0,zhongnanhai,location,my dataframe2
1,cats,pop culture,my dataframe2
2,zhongnanhai,location,my dataframe2


In [25]:
weibo.run(more_keywords_df,sqlite_file=sqlite_file,cookies=cookie)

(0, u'zhongnanhai', 'has_results')
(1, u'cats', 'has_results')


In [26]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None
3,3,2017-09-25,2017-09-25 10:14:29.667395,1,刘晓波,0,0,0,censored,my dataframe,NaN,None
4,4,2017-09-25,2017-09-25 10:14:49.107078,1,pokemon,0,0,0,has_results,my dataframe,5705260.0,pop culture
5,5,2017-09-25,2017-09-25 10:15:09.762484,1,jay chou,0,0,0,has_results,my dataframe,881.0,pop culture
6,6,2017-09-25,2017-09-25 10:15:28.100418,1,weibo,0,0,0,has_results,my dataframe,63401495.0,social media
7,7,2017-09-25,2017-09-25 10:15:46.214464,2,hello,0,0,0,has_results,my dataframe,80454634.0,None
8,8,2017-09-25,2017-09-25 10:16:03.274804,2,lxb,0,0,0,censored,my dataframe,NaN,None
9,9,2017-09-25,2017-09-25 10:16:19.035805,2,习胞子,0,0,0,no_results,my dataframe,NaN,None


## You can also pass in lists if you prefer (though you can't include the source or notes)

In [27]:
sample_keywords_list = ["cats",'yes','自由亚洲电台','刘晓波','dhfjkdashfjkasdhf']

**See below how it handles connection reset errors (it waits a little extra to make sure your connection clears before continuing testing)**

In [28]:
weibo.run(sample_keywords_list,sqlite_file=sqlite_file,cookies=cookie)

(0, u'cats', 'has_results')
(1, u'yes', 'has_results')
自由亚洲电台 caused connection reset, waiting 95
(2, u'\u81ea\u7531\u4e9a\u6d32\u7535\u53f0', 'reset')
(3, u'\u5218\u6653\u6ce2', 'censored')
(4, u'dhfjkdashfjkasdsf87', 'no_results')


In [29]:
weibo.sqlite_to_df(sqlite_file)

,id,date,datetime_logged,test_number,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-09-25,2017-09-25 10:13:37.816720,1,hello,0,0,0,has_results,my dataframe,80454701.0,None
1,1,2017-09-25,2017-09-25 10:13:54.356722,1,lxb,0,0,0,censored,my dataframe,NaN,None
2,2,2017-09-25,2017-09-25 10:14:11.489530,1,习胞子,0,0,0,no_results,my dataframe,NaN,None
3,3,2017-09-25,2017-09-25 10:14:29.667395,1,刘晓波,0,0,0,censored,my dataframe,NaN,None
4,4,2017-09-25,2017-09-25 10:14:49.107078,1,pokemon,0,0,0,has_results,my dataframe,5705260.0,pop culture
5,5,2017-09-25,2017-09-25 10:15:09.762484,1,jay chou,0,0,0,has_results,my dataframe,881.0,pop culture
6,6,2017-09-25,2017-09-25 10:15:28.100418,1,weibo,0,0,0,has_results,my dataframe,63401495.0,social media
7,7,2017-09-25,2017-09-25 10:15:46.214464,2,hello,0,0,0,has_results,my dataframe,80454634.0,None
8,8,2017-09-25,2017-09-25 10:16:03.274804,2,lxb,0,0,0,censored,my dataframe,NaN,None
9,9,2017-09-25,2017-09-25 10:16:19.035805,2,习胞子,0,0,0,no_results,my dataframe,NaN,None
